**Motifs computation using a Greedy Algorithm, Greedy Algorithm with pseudocounts and Randomized Motif Search Algorithm**

Some genes, that encode transription factors are able to control other genes. These have the ability to turn other genes on and off. A TF are capable of this regulation by binding to a specific place- transcription binding factor or in other words regulatory motif.

Hence, the goal of this project is to develop algorithms finding these places in a genome.

First, lets write a function that takes a list of strings (motifs) and counts the number of occurances of nucleotides on each position in a string- returning a count Matrix.

In [167]:
def Count(Motifs):
    count = {} 
    k = len(Motifs[0])
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
             count[symbol].append(0)
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

Then, we are computing the profile of a motif- constructing the matrix of i-th nucleotide in the j-th column.

In [168]:
def Profile(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = Count(Motifs)
    for key,value in profile.items():
        value[:]=[x/t for x in value]
        
    return profile

Once we know what are the most frequently occuring nucleotides on a particular position, we contruct a Consensus string.

In [169]:
def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

As the last task, we are calculating the score which is a quantity of nucleotides, that did not fall into the Consensus string.

In [170]:
def Score(Motifs):
    consensus = Consensus(Motifs)
    count = 0
    for motif in Motifs:
        for symbol in range(len(motif)):
            if motif[symbol] != consensus[symbol]:
                count += 1
    return count

The goal is to find a set of k-mers and at the same time minimizing the score, when having a collection of strings as the input.

We will approach this problem with greedy algorithm, which is more effective than brute force solution. In order to do this, we will first define a probability function, that given a String and earlier calculated profile matrix- computes the probability of the occurance of such motif.

In [171]:
def Pr(Text, profile):
    p=1
    for i in range(len(Text)):
        p *= profile[Text[i]][i]
    return p

Now, we are defining a function for profile most probable Kmer, that computes it, given a String, k (length of kmer that we are allowing) and profile.

In [172]:
def ProfileMostProbableKmer(Text, k, profile):
    patternProb={}
    k_mer=[]
    for i in range(len(Text)-k+1):
        k_mer=Text[i:i+k]
        Probability=Pr(k_mer,profile)
        patternProb[k_mer]=Probability
        
    m = max(patternProb.values())
    max_Prob=[]
    for key in patternProb:
        if patternProb[key]==m:
            max_Prob.append(key)
        
    return max_Prob[0]

To wrap it all up, lets write a function that uses the previous knowledge and functions and based on that returns best possible motifs.

In [173]:
def GreedyMotifSearch(Dna, k, t):
    
    BestMotifs=[]
    for i in range(0,t):
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

Now, lets test it on dormancy survival regulator (DosR) transcription factor. As an input DNA, there are 10 DNA chunks- each of 250 nucleotides long (genes), that during hypoxic conditions had changed their expression levels, which could lead to suspecting them to have a regulatory motif within.

In [174]:
Dna = [
"GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
"CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
"ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC", 
"GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
"GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG", 
"CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA", 
"GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA", 
"GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG", 
"GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG", 
"TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"
]

Lets set t- number of kmers in DNA to quantity of the input strings (10) and k to 15.

In [175]:
t=len(Dna)
k=15

and calculate the Motifs obtained with GreedyMotifSearch funcion

In [176]:
Motifs=GreedyMotifSearch(Dna, k, t)
print(Motifs)

['GTTAGGGCCGGAAGT', 'CCGATCGGCATCACT', 'ACCGTCGATGTGCCC', 'GGGTCAGGTATATTT', 'GTGACCGACGTCCCC', 'CTGTTCGCCGGCAGC', 'CTGTTCGATATCACC', 'GTACATGTCCAGAGC', 'GCGATAGGTGAGATT', 'CTCATCGCTGTCATC']


Lastly, lets print out the score of the obtained Motifs.

In [177]:
print(Score(Motifs))

64


Although GreedyMotifSearch looks like a perfect solution, it does have its flaws. It's enough to have one value in a profile for a given sequence equals 0, to obtain meaningless value for its probability- also 0. That is why, it is useful to introduce pseudocounts to replace 0 in profile with really small values. One way of doing it is with Laplace's rule of succession- where we are adding one to each value in a profile. In order to do that we start with changing the count function- initializing the count of a given symbol with 1 instead of 0.

In [178]:
def CountWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    count = {}
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(1)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

Now, lets change the Profile function accordingly- we added 4 in total- one for each symbol. We are adjusting our computing respectively.

In [179]:
def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = CountWithPseudocounts(Motifs)
    for key,value in profile.items():
        value[:]=[x/(t+4) for x in value]
        
    return profile

We have to consider our new function with pseudocounts also in a Consensus string calculation.

In [180]:
def Consensus(Motifs):
    k = len(Motifs[0])
    count = CountWithPseudocounts(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

Putting it all together, we are adjusting our GreedyMotifSearch function with new pseudocounts subroutines.

In [181]:
def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    
    BestMotifs=[]
    for i in range(0,t):
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = ProfileWithPseudocounts(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

Next step is to calculate the Motifs given a pseudocounts considerations.

In [182]:
MotifsWithPseudocounts=GreedyMotifSearchWithPseudocounts(Dna, k, t)
print(MotifsWithPseudocounts)

['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGATTACCGACCGCA', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GGACCGAAGTCCCCG']


In [183]:
print(Score(MotifsWithPseudocounts))

35


In [184]:
print(Score(Motifs))

64


After printing out the score of Motifs With Pseudocounts and without this consideration, we see that the results are almost two times better (smaller score) for the outcome given pseudocounts.

Next approach would be to incorporate randomized algorithms into our reasoning.

We are starting with defining a function which takes a Profile and list of DNA strings and returns most probable kmers for each of them.

In [185]:
def Motifs(Profile, Dna):
    Motifs = []
    t = len(Dna)
    k = len(Profile['A'])
    for i in range(t):
        Motif = ProfileMostProbableKmer(Dna[i], k, Profile)
        Motifs.append(Motif)
    return Motifs

Then, we are introducing randomness to our motif search.

In [186]:
import random
def RandomMotifs(Dna, k, t):
    t = len(Dna)
    n = len(Dna[0])
    strings = []
    for i in range(t):
        randnum = random.randint(0,n-k)
        strings.append(Dna[i][randnum:randnum+k])
    return strings

In [187]:
def RandomizedMotifSearch(Dna,k,t):
    M = RandomMotifs(Dna, k, t)
    BestMotifs = M
    while True:
        Profile = ProfileWithPseudocounts(M)
        M = Motifs(Profile, Dna)
        if Score(M) < Score(BestMotifs):
            BestMotifs = M
        else:
            return BestMotifs

Last step is to test it with N=100 runs on our DosR DNA strings.

In [188]:
t = len(Dna)
k=15
N=100

In [189]:
BestMotifs = RandomizedMotifSearch(Dna, k, t)
for i in range(N):
    m = RandomizedMotifSearch(Dna, k, t)
    if Score(m) < Score(BestMotifs):
        BestMotifs = m
    

In [190]:
print(BestMotifs)

['GGGCCGGAAGTCCCC', 'GGGTCAAACGACCCT', 'GGGACGTAAGTCCCT', 'GCGCCAGAATCCCGG', 'GGGGCGAAAGTCCCT', 'GCGGCGGTGGTCCGG', 'GGGTCAATGGTCCCC', 'GGGACTTTCGGCCCT', 'GGGACCAACGCCCCT', 'GGGACCGAAGTCCCC']


In [191]:
print(Score(BestMotifs))

40


We got a score of 40, which is a very close result to the one obtained with greedy motif search with pseudocounts. By adjusting the N parameter we can achieve even better results.